**<font color = black size=6>实验九:贝叶斯分类</font>**

In [113]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

**<font color = blue size=4>第一部分:实验任务</font>**

1.朴素贝叶斯

<img src='./Naive Bayes Classifier Pseudocode.jpg'>

<span style="color:purple">该数据集(train_mushroom.csv)为分类数据集，为蘑菇的特征信息以及是否有毒。包括了13个特征以及一个标签(即为label类型,代表是否有毒)。label='p'代表有毒，label='e'代表无毒。</span>

<span style="color:purple">1) 将训练数据集'train_mushroom.csv'和'test_mushroom.csv'载入并转换为你需要的格式</span>

In [114]:
# 训练集 将数据格式转换为numpy数组
train_frame = pd.read_csv("train_mushroom.csv")
train_data = np.array(train_frame)
# 测试集 将数据格式转换为numpy数组
test_frame = pd.read_csv("test_mushroom.csv")
test_data = np.array(test_frame)
print(train_data)
print(test_data)

[['s' 'n' 't' ... 'u' 'x' 'p']
 ['s' 'y' 't' ... 'g' 'x' 'e']
 ['s' 'w' 't' ... 'm' 'b' 'e']
 ...
 ['f' 'y' 'y' ... 'v' 'd' 'p']
 ['f' 'y' 'y' ... 'v' 'p' 'p']
 ['f' 'y' 'y' ... 'y' 'p' 'p']]
[['k' 'y' 'n' ... 'v' 'd' 'p']
 ['b' 'f' 'g' ... 's' 'g' 'e']
 ['b' 's' 'g' ... 'n' 'g' 'e']
 ...
 ['k' 's' 'n' ... 'v' 'l' 'p']
 ['f' 'y' 'n' ... 'v' 'd' 'p']
 ['x' 's' 'w' ... 's' 'g' 'e']]


<span style="color:purple">2) 计算每个标签值y对应的先验概率P(y)
$$P(y)=\frac{|D_y|}{|D|}$$
    其中$D_y$为标签值为y的样本集合，$|D_y|$为这个集合的样本个数；D为所有样本集合，|D|为所有样本个数

</span>

In [115]:
# 计算先验概率
m = train_data.shape[0]  # 训练集样本个数
unique_train_labels = np.unique(train_data[:, -1])
print("未引入拉普拉斯平滑 标签值和对应的先验概率")
print(unique_train_labels)  # 打印查看标记种类
label_counter = Counter(train_data[:, -1])  # 存储标签key和对应的个数
P = []  # 存储先验概率
# 计算每个标签的先验概率并进行存储
for label in unique_train_labels:
    P.append(label_counter[label] / m)
print(P)
prior_prob_e = P[0]  # 存储标签e的先验概率
prior_prob_p = P[1]  # 存储标签p的先验概率

未引入拉普拉斯平滑 标签值和对应的先验概率
['e' 'p']
[0.59, 0.41]


<span style="color:purple">3) 对于数据集中的每个特征的非重复特征值$x_i$，计算给定标签值y时特征值$x_i$的条件概率$P(x_i│y)$,
    $$P(x_i│y)=\frac{|D_{x_i,y}|}{|D_y|}$$
$D_{x_i,y}$为标签值为y，特征值为$x_i$的样本集合；$|D_{x_i,y}|$为该集合的样本个数
</span>

In [116]:
# 首先遍历数据集D中的每个特征，将每个特征的非重复值取出
m = train_data.shape[0]  # 样本个数
n = train_data.shape[1] - 1  # 属性个数

unique_features = []  # 存放每个属性的非重复值
for i in range(n):  # l[i]表示第i个属性的非重复值
    counter = Counter(train_data[:, i])
    unique_features.append(list(counter))
print(unique_features)  # 打印查看数据集中每个特征的非重复值

# 根据标签值将数据集D分为两个子数据集，分别包括所有标签值为p的样本和所有标签值为e的样本。
train_data_e = train_data[train_data[:, -1] == "e"]
train_data_p = train_data[train_data[:, -1] == "p"]
# 现以标签为p的数据集Dp为例子，遍历Dp的每个特征，分别求出该特征每个特征值的条件概率
# 以特征cap-shape为例。Dp中cap-shape的非重复值集合为['b' 'c' 'f' 'k' 's' 'x' 'y']，计算条件概率P(cap-shape='b'|label='p'),P(cap-shape='c'|label='p'),...,P(cap-shape='y'|label='p')，
# 上述对cap-shape特征操作完成后，按照同样的步骤对Dp中的剩余12个特征进行同样的操作

cond_probabilities_p = {}  # 存储标记为p的条件概率结果
for k in range(train_data_p.shape[1] - 1):
    cond_probabilities_p[k] = {}  # 初始化 将对应每个属性索引处新建一个字典
# 遍历子数据集的全部特征
for j in range(train_data_p.shape[1] - 1):
    # 遍历全部样本
    feature_unique = unique_features[j]  # 获取每个属性的非重复值
    for value in feature_unique:
        # 对于每个属性 重新从全部样本中进行寻找
        count_label = 0
        for i in range(len(train_data_p)):
            if train_data_p[i][j] == value:
                count_label += 1
        # 在对应属性列的位置 设置对应的字典k-v值 即 属性值-条件概率值
        temple_dict = {}
        temple_dict[value] = count_label / len(train_data_p)
        cond_probabilities_p[j].update(temple_dict)
print(cond_probabilities_p)

# 得到数据集Dp中条件概率集合后，对另一个子数据集进行同样的操作

cond_probabilities_e = {}  # 存储标记为e的条件概率结果
for k in range(train_data_e.shape[1] - 1):
    cond_probabilities_e[k] = {}
# 遍历子数据集的全部特征
for j in range(train_data_e.shape[1] - 1):
    # 遍历全部样本
    feature_unique = unique_features[j]  # 获取每个属性的非重复值
    for value in feature_unique:
        # 对于每个属性 重新从全部样本中进行寻找
        count_label = 0
        for i in range(len(train_data_e)):
            if train_data_e[i][j] == value:
                count_label += 1
        # 在对应属性列的位置 设置对应的字典k-v值 即 属性值-条件概率值
        temple_dict = {}  # 创建的临时字典 存储当前属性列的属性的条件概率
        temple_dict[value] = count_label / len(train_data_e)
        cond_probabilities_e[j].update(temple_dict)
print(cond_probabilities_e)

[['s', 'y', 'f', 'x', 'b', 'k', 'c'], ['n', 'y', 'w', 'g', 's', 'f'], ['t', 'f', 'w', 'n', 'p', 'e', 'g', 'y', 'b', 'r', 'c', 'u'], ['p', 'a', 'l', 'n', 't', 'f'], ['f', 'y', 'n', 's'], ['n', 'b', 'c', 'w'], ['k', 'n', 'g', 'p', 'w', 'h', 'b'], ['e', 't', 'w', 'p', 'b', 'g', 'h', 'r'], ['p', 'e', 'o', 't'], ['k', 'n', 'u', 'p', 'e', 'l', 'f'], ['s', 'n', 'a', 'v', 'y', 'h', 'w', 'r'], ['u', 'g', 'm', 'd', 'p', 'v', 's', 'c', 'y'], ['x', 'b', 's', 'f', 'u', 'd', 'g', 'l', 'p', 'm', 'w']]
{0: {'s': 0.05853658536585366, 'y': 0.11219512195121951, 'f': 0.36097560975609755, 'x': 0.424390243902439, 'b': 0.03902439024390244, 'k': 0.0, 'c': 0.004878048780487805}, 1: {'n': 0.06341463414634146, 'y': 0.5024390243902439, 'w': 0.1073170731707317, 'g': 0.0, 's': 0.2682926829268293, 'f': 0.05853658536585366}, 2: {'t': 0.17073170731707318, 'f': 0.00975609756097561, 'w': 0.14146341463414633, 'n': 0.17560975609756097, 'p': 0.02926829268292683, 'e': 0.04390243902439024, 'g': 0.12682926829268293, 'y': 0.21

<span style="color:purple">4) 编写函数，给定样本$x=(x_1,...,x_i,...,x_d)$以及标签y, 计算其后验概率    
    输入：样本x，标签y  
    输出：样本x对应标签y的后验概率  
    计算后验概率公式:
$P(y)\prod_{i=1}^{d}P(x_i|y)$   
    
<span style="color:purple">例:  
    特征和标签：(cap-shape, cap-surface,..., habitat), label  
    输入: [k, y, n, f, s, c, n, b, o, e, w, v, d], p  
    输出: P(label='p') $\times$ P(cap-shape='k'|label='p') $\times$ ... $\times$ P(habitat='d'|label='p')</span>

In [117]:
# 经过测试发现测试集中存在新的属性值 导致字典出现键值对不匹配的错误
# 因此选择对该错误捕捉并进行相应处理
def pro(a, index):
    # 初始化后验概率为之前计算的先验概率
    post_pro = prior_prob_e if index == "e" else prior_prob_p
    for i in range(len(a)):
        try:
            if index == "p":
                # print(f"处理特征 {i} 的值 {a[i]}")  # 打印当前处理的特征
                # 乘以对应i索引属性列下的属性值a[i]的条件概率
                post_pro *= cond_probabilities_p[i][a[i]]
            else:
                # print(f"处理特征 {i} 的值 {a[i]}")  # 打印当前处理的特征
                # 乘以对应i索引属性列下的属性值a[i]的条件概率
                post_pro *= cond_probabilities_e[i][a[i]]
        except KeyError:
            # print(f"出现未知特征值: 第{i}列，值 {a[i]}")  # 打印异常信息
            # 直接忽略未知的特征值
            pass
    return post_pro

def pro_new(a, index):
    # 初始化后验概率为之前计算的先验概率
    post_pro = prior_prob_e if index == "e" else prior_prob_p
    for i in range(len(a)):
        try:
            if index == "p":
                # print(f"处理特征 {i} 的值 {a[i]}")  # 打印当前处理的特征
                # 乘以对应i索引属性列下的属性值a[i]的条件概率
                post_pro *= cond_probabilities_p[i][a[i]]
            else:
                # print(f"处理特征 {i} 的值 {a[i]}")  # 打印当前处理的特征
                # 乘以对应i索引属性列下的属性值a[i]的条件概率
                post_pro *= cond_probabilities_e[i][a[i]]
        except KeyError:
            print(f"出现未知特征值: 第{i}列，值 {a[i]}")  # 打印异常信息
            # 直接忽略未知的特征值
            post_pro = 0
            pass
    return post_pro

<span style="color:purple">5) 对测试集中的每个样本a，利用上个步骤所编写的函数，分别求所有可能的标签对应的后验概率，后验概率更大的对应标签即为预测标签。最后与测试集本身标签进行比较计算出准确率</span>

In [118]:
# true_labels = test_data[:, -1]  # 测试集的真实标签
# pre_labels = []  # 预测的标签
# # 遍历测试集中全部样本
# for i in range(test_data.shape[0]):
#     pro_e = pro(test_data[i, :-1], "e")
#     pro_p = pro(test_data[i, :-1], "p")
#     if pro_e == 0 and pro_p == 0:
#         print("----------------------")
#     temple_label = "e" if pro_e > pro_p else "p"  # 通过后验概率的大小判断预测标签
#     pre_labels.append(temple_label)  # 添加预测的标签
# # 计算准确率
# accuracy = np.mean(pre_labels == true_labels)
# print("准确率为: {:.2f}%".format(accuracy * 100))

# true_labels = test_data[:, -1]  # 测试集的真实标签
# invalid_count = 0  # 统计无法预测的样本的数量
# correct_count = 0  # 统计正确预测的样本的数量
# # 遍历测试集中全部样本
# for i in range(test_data.shape[0]):
#     pro_e = pro(test_data[i, :-1], "e")
#     pro_p = pro(test_data[i, :-1], "p")
#     # 没有引入拉普拉斯平滑 存在可能都为0的情况
#     if pro_e == pro_p:
#         invalid_count += 1  # 统计无法判别的样本个数
#     else:
#         temple_label = "e" if pro_e > pro_p else "p"  # 通过后验概率的大小判断预测标签
#         # 统计预测正确的个数
#         if temple_label == true_labels[i]:
#             correct_count += 1
# # 计算准确率
# accuracy = correct_count / (len(true_labels) - invalid_count)
# print("准确率为: {:.2f}%".format(accuracy * 100))


def test_function(pro_function):
    # 定义测试函数 传入不同的后验概率计算函数
    true_labels = test_data[:, -1]  # 测试集的真实标签
    invalid_count = 0  # 统计无法预测的样本的数量
    correct_count = 0  # 统计正确预测的样本的数量
    # 遍历测试集中全部样本
    for i in range(test_data.shape[0]):
        pro_e = pro_function(test_data[i, :-1], "e")
        pro_p = pro_function(test_data[i, :-1], "p")
        # 没有引入拉普拉斯平滑 存在可能都为0的情况
        # 计算后验概率的函数也可能直接将后验概率置为0
        # 同时 若后验概率大小相等 则该样本也不能被区分
        if pro_e == pro_p:
            invalid_count += 1  # 统计无法判别的样本个数
        else:
            temple_label = "e" if pro_e > pro_p else "p"  # 通过后验概率的大小判断预测标签
            # 统计预测正确的个数
            if temple_label == true_labels[i]:
                correct_count += 1
    # 计算准确率
    accuracy = correct_count / (len(true_labels) - invalid_count)
    print("准确率为: {:.2f}%".format(accuracy * 100))


print("朴素贝叶斯")
print()
print("忽略未知的特征值")
test_function(pro)
print()
print("后验概率置为0")
test_function(pro_new)

朴素贝叶斯

忽略未知的特征值
准确率为: 21.05%

后验概率置为0
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第4列，值 m
出现未知特征值: 第7列，值 y
出现未知特征值: 第8列，值 n
出现未知特征值: 第4列，值 m
出现未知特征值: 第7列，值 y
出现未知特征值: 第8列，值 n
出现未知特征值: 第7列，值 y
出现未知特征值: 第7列，值 y
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 y
出现未知特征值: 第7列，值 y
出现未知特征值: 第7列，值 y
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 y
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 n
出现未知特征值: 第7列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第7列，值 y
出现未知特征值: 第7列，值 y
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 o
出现未知特征值: 第7列，

2.引入拉普拉斯平滑

<span style="color:purple">1) 首先，请判断是否有某个特征值和某个类没有在训练集中同时出现而使得条件概率为零。如果无，则无需进行下列实验；如果有，请在上个实验的基础上引入拉普拉斯平滑</span>

In [119]:
# 设置初始标签Flag来进行判定
Flag_e = False
Flag_p = False
# 遍历全部属性n
for i in range(n):
    # 遍历每个属性列下的全部的键值对
    for j in cond_probabilities_e[i]:
        # 检查某个属性值的条件概率是否为0
        if cond_probabilities_e[i][j] == 0:
            Flag_e = True
            break
    for j in cond_probabilities_p[i]:
        # 通过其值是否为0来判断
        if cond_probabilities_p[i][j] == 0:
            Flag_p = True
            break
if Flag_e or Flag_p:
    print("存在条件概率为0的情况,需要引入拉普拉斯平滑")
else:
    print("不存在条件概率为0的情况,不需要引入拉普拉斯平滑")

存在条件概率为0的情况,需要引入拉普拉斯平滑


<span style="color:purple">2) 引入拉普拉斯平滑后计算每个标签y对应的先验概率P(y)
$$P(y)=\frac{|D_y|+1}{|D|+N}$$
    其中$D_y$为标签为y的样本集合；$|D_y|$为标签为y的集合的样本个数；D为所有样本集合；|D|为所有样本个数;N为标签取值的个数

</span>

In [120]:
# 计算先验概率
P = []
for label in unique_train_labels:
    D_y_1 = label_counter[label] + 1
    D_N = train_data.shape[0] + len(list(label_counter))
    P.append(D_y_1 / D_N)
print("引入拉普拉斯平滑 标签值和对应的先验概率")
print(unique_train_labels)
print(P)
prior_prob_e2 = P[0]
prior_prob_p2 = P[1]
# print(prior_prob_e2)
# print(prior_prob_p2)

引入拉普拉斯平滑 标签值和对应的先验概率
['e' 'p']
[0.5896414342629482, 0.4103585657370518]


<span style="color:purple">3) 计算数据集中的每个特征的非重复特征值$x_i$对应标签y的条件概率$P(x_i│y)$,
    $$P(x_i│y)=\frac{|D_{x_i,y}|+1}{|D_y|+N_i}$$
$D_{x_i,y}$为标签为$y$，特征为$x_i$的样本集合；$|D_{x_i,y}|$为该样本个数;$N_i$为第$i$个特征取值的个数
</span>

In [121]:
# 首先遍历数据集D中的每个特征，将每个特征的非重复值取出
n = train_data.shape[1] - 1  # 属性个数
m = train_data.shape[0]  # 样本个数
label_loc = train_data.shape[1] - 1  # 定位标签的索引

unique_features = []  # 存放每个属性的非重复值
for i in range(n):  # l[i]表示第i个属性的非重复值
    counter = Counter(train_data[:, i])
    val = list(counter)
    unique_features.append(val)
print(unique_features)  # 打印查看数据集中每个特征的非重复值

# 根据标签值将数据集D分为两个子数据集，分别包括所有标签值为p的样本和所有标签值为e的样本。

train_data_e = train_data[train_data[:, label_loc] == "e"]
train_data_p = train_data[train_data[:, label_loc] == "p"]

# 现以标签为p的数据集Dp为例子，遍历Dp的每个特征，分别求出该特征每个特征值的条件概率。
# 以特征cap-shape为例。Dp中cap-shape的非重复值集合为['b' 'c' 'f' 'k' 's' 'x' 'y']，计算条件概率P(cap-shape='b'|label='p'),P(cap-shape='c'|label='p'),...,P(cap-shape='y'|label='p')，
# 计算条件概率时需要注意引入拉普拉斯平滑。
# 上述对cap-shape特征操作完成后，按照同样的步骤对Dp中的剩余12个特征进行同样的操作
cond_probabilities_p = {}  # 存储标记为p的条件概率结果
for k in range(train_data_p.shape[1] - 1):
    cond_probabilities_p[k] = {}  # 初始化 将对应每个属性索引处新建一个字典

# 遍历子数据集的全部特征
for j in range(train_data_p.shape[1] - 1):
    # 遍历全部样本
    feature_unique = unique_features[j]  # 获取每个属性的非重复值
    for value in feature_unique:
        # 对于每个属性 重新从全部样本中进行寻找
        count_label = 0
        for i in range(len(train_data_p)):
            if train_data_p[i][j] == value:
                count_label += 1
        # 在对应属性列的位置 设置对应的字典k-v值 即 属性值-条件概率值
        temple_dict = {}
        temple_dict[value] = (count_label + 1) / (len(train_data_p) + len(feature_unique))
        cond_probabilities_p[j].update(temple_dict)
print(cond_probabilities_p)

# 得到数据集Dp中条件概率集合后，对另一个子数据集进行同样的操作
cond_probabilities_e = {}  # 存储标记为e的条件概率结果
for k in range(train_data_e.shape[1] - 1):
    cond_probabilities_e[k] = {}
# 遍历子数据集的全部特征
for j in range(train_data_e.shape[1] - 1):
    # 遍历全部样本
    feature_unique = unique_features[j]  # 获取每个属性的非重复值
    for value in feature_unique:
        # 对于每个属性 重新从全部样本中进行寻找
        count_label = 0
        for i in range(len(train_data_e)):
            if train_data_e[i][j] == value:
                count_label += 1
        # 在对应属性列的位置 设置对应的字典k-v值 即 属性值-条件概率值
        temple_dict = {}  # 创建的临时字典 存储当前属性列的属性的条件概率
        temple_dict[value] = (count_label + 1) / (len(train_data_e) + len(feature_unique))
        cond_probabilities_e[j].update(temple_dict)
print(cond_probabilities_e)

[['s', 'y', 'f', 'x', 'b', 'k', 'c'], ['n', 'y', 'w', 'g', 's', 'f'], ['t', 'f', 'w', 'n', 'p', 'e', 'g', 'y', 'b', 'r', 'c', 'u'], ['p', 'a', 'l', 'n', 't', 'f'], ['f', 'y', 'n', 's'], ['n', 'b', 'c', 'w'], ['k', 'n', 'g', 'p', 'w', 'h', 'b'], ['e', 't', 'w', 'p', 'b', 'g', 'h', 'r'], ['p', 'e', 'o', 't'], ['k', 'n', 'u', 'p', 'e', 'l', 'f'], ['s', 'n', 'a', 'v', 'y', 'h', 'w', 'r'], ['u', 'g', 'm', 'd', 'p', 'v', 's', 'c', 'y'], ['x', 'b', 's', 'f', 'u', 'd', 'g', 'l', 'p', 'm', 'w']]
{0: {'s': 0.06132075471698113, 'y': 0.11320754716981132, 'f': 0.35377358490566035, 'x': 0.41509433962264153, 'b': 0.04245283018867924, 'k': 0.0047169811320754715, 'c': 0.009433962264150943}, 1: {'n': 0.06635071090047394, 'y': 0.4928909952606635, 'w': 0.10900473933649289, 'g': 0.004739336492890996, 's': 0.26540284360189575, 'f': 0.061611374407582936}, 2: {'t': 0.16589861751152074, 'f': 0.013824884792626729, 'w': 0.1382488479262673, 'n': 0.17050691244239632, 'p': 0.03225806451612903, 'e': 0.04608294930875

In [122]:
# 经过测试发现测试集中存在新的属性值 导致字典出现键值对不匹配的错误
# 因此选择对该错误捕捉并进行相应处理
def pro2(a, index):
    # 初始化后验概率为之前计算的先验概率
    post_pro = prior_prob_e2 if index == "e" else prior_prob_p2
    try:
        #  所给标记为p
        if index == "p":
            for i in range(len(a)):
                # 乘以对应i索引属性列下的属性值a[i]的条件概率
                post_pro *= cond_probabilities_p[i][a[i]]
        else:  # 所给标记为e
            for i in range(len(a)):
                # 乘以对应i索引属性列下的属性值a[i]的条件概率
                post_pro *= cond_probabilities_e[i][a[i]]
    except KeyError:
        # 测试集中出现新的属性值 选择是否将其条件概率置为0 即将整体的后验概率置为0
        post_pro = 0
        # print("出现训练集中未有的属性值:第{}列属性值{}".format(i, a[i]))
        pass
    # 返回计算出的后验概率
    return post_pro

<span style="color:purple">4) 对测试集中的每个样本a，利用上个步骤所编写的函数，分别求所有可能的标签对应的后验概率，后验概率更大的对应标签即为预测标签，最后与测试集本身标签进行比较计算出准确率</span>

In [123]:
true_labels = test_data[:, -1]  # 测试集的真实标签
pre_labels = []  # 预测的标签
invalid_count = 0
correct_count = 0
# 遍历测试集中全部样本
for i in range(test_data.shape[0]):
    pro_e = pro2(test_data[i, :-1], "e")
    pro_p = pro2(test_data[i, :-1], "p")
    if pro_e == pro_p:
        invalid_count += 1  # 统计无法判别的样本个数
    else:
        temple_label = "e" if pro_e > pro_p else "p"  # 通过后验概率的大小判断预测标签
        if temple_label == true_labels[i]:
            correct_count += 1
    # pre_labels.append(temple_label)  # 添加预测的标签
# 计算准确率
# accuracy = np.mean(pre_labels == true_labels)
print(len(true_labels))
print(invalid_count)
accuracy = correct_count / (len(true_labels) - invalid_count)
print("准确率为: {:.2f}%".format(accuracy * 100))

100
22
准确率为: 70.51%


In [124]:
# 忽略未知特征值
def pro3(a, index):
    # 初始化后验概率为之前计算的先验概率
    post_pro = prior_prob_e2 if index == "e" else prior_prob_p2
    for i in range(len(a)):
        try:
            if index == "p":
                # print(f"处理特征 {i} 的值 {a[i]}")  # 打印当前处理的特征
                # 乘以对应i索引属性列下的属性值a[i]的条件概率
                post_pro *= cond_probabilities_p[i][a[i]]
            else:
                # print(f"处理特征 {i} 的值 {a[i]}")  # 打印当前处理的特征
                # 乘以对应i索引属性列下的属性值a[i]的条件概率
                post_pro *= cond_probabilities_e[i][a[i]]
        except KeyError:
            # print(f"出现未知特征值: 第{i}列，值 {a[i]}")  # 打印异常信息
            # 直接忽略未知的特征值
            pass
    return post_pro

In [125]:
# 后验概率置为0
def pro4(a, index):
    # 初始化后验概率为之前计算的先验概率
    post_pro = prior_prob_e2 if index == "e" else prior_prob_p2
    for i in range(len(a)):
        try:
            if index == "p":
                # print(f"处理特征 {i} 的值 {a[i]}")  # 打印当前处理的特征
                # 乘以对应i索引属性列下的属性值a[i]的条件概率
                post_pro *= cond_probabilities_p[i][a[i]]
            else:
                # print(f"处理特征 {i} 的值 {a[i]}")  # 打印当前处理的特征
                # 乘以对应i索引属性列下的属性值a[i]的条件概率
                post_pro *= cond_probabilities_e[i][a[i]]
        except KeyError:
            print(f"出现未知特征值: 第{i}列，值 {a[i]}")  # 打印异常信息
            post_pro = 0  # 将后验概率置为0
    return post_pro

In [126]:
test_unique_features = []  # 存放每个属性的非重复值
for i in range(n):  # l[i]表示第i个属性的非重复值
    counter = Counter(test_data[:, i])
    val = list(counter)
    test_unique_features.append(val)

# 平滑处理
def pro5(a, index):

    # 初始化后验概率为之前计算的先验概率

    post_pro = prior_prob_e2 if index == "e" else prior_prob_p2

    for i in range(len(a)):

        try:

            if index == "p":

                # print(f"处理特征 {i} 的值 {a[i]}")  # 打印当前处理的特征

                # 乘以对应i索引属性列下的属性值a[i]的条件概率

                post_pro *= cond_probabilities_p[i][a[i]]

            else:

                # print(f"处理特征 {i} 的值 {a[i]}")  # 打印当前处理的特征

                # 乘以对应i索引属性列下的属性值a[i]的条件概率

                post_pro *= cond_probabilities_e[i][a[i]]

        except KeyError:

            # print(f"出现未知特征值: 第{i}列，值 {a[i]}")  # 打印异常信息

            post_pro *= 1 / len(unique_features[i])  # 将条件概率设置为测试集单一属性的概率
    return post_pro



print("引入拉普拉斯平滑后")
print()

print("后验概率置为0")

test_function(pro4)
print()

print("忽略未知特征值")

test_function(pro3)

引入拉普拉斯平滑后

后验概率置为0
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第4列，值 m
出现未知特征值: 第7列，值 y
出现未知特征值: 第8列，值 n
出现未知特征值: 第4列，值 m
出现未知特征值: 第7列，值 y
出现未知特征值: 第8列，值 n
出现未知特征值: 第7列，值 y
出现未知特征值: 第7列，值 y
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 y
出现未知特征值: 第7列，值 y
出现未知特征值: 第7列，值 y
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 y
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 n
出现未知特征值: 第7列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 b
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第11列，值 n
出现未知特征值: 第7列，值 y
出现未知特征值: 第7列，值 y
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值 o
出现未知特征值: 第7列，值 n
出现未知特征值: 第10列，值

**<font color = blue size=4>第二部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可，如果交错请到讲台前联系助教，删掉之前的错误版本后再进行提交

二、实验报告截止日期： 【11月17日 14:20】
要求：  
1)文件格式为：学号-姓名.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  
3)文件命名时不需要额外添加“实验几”等额外信息，按照格式提交  
4)每周的实验报告提交地址会变化，且有时间限制，提交时间为下周的实验课开始时，请注意及时提交。

实验九(贝叶斯分类)的实验报告上交地址:https://send2me.cn/ufVNphux/T9yuatQDc00TVw  

三、课堂课件获取地址:https://www.jianguoyun.com/p/DRLiP2oQp5WhChjB86YFIAA  
实验内容获取地址:https://www.jianguoyun.com/p/DbLessAQp5WhChjD86YFIAA